# Santas Helper Demo

Quick demo to show how the allocation logic works.

In [ ]:
# Setup the variables that are specific to your environment (you will need to change this! :-)
project_id='mv-santas-helper'
dataset_id='santas_helper'
service_acc_file='configuration.json'

In [ ]:
# Install any dependencies used (set in the requirements.txt file)
!pip install -r requirements.txt

In [ ]:
# Load the helper library (primarily for the big query connection object)
from lib import *

In [ ]:
# Setup the bigquery connection object using your credentials
b = bq(project_id=project_id, credentials_file=service_acc_file)

In [ ]:
# Check the connection by reading the child/present scores table
childPresentScores = b.q(f'''
SELECT * 
FROM `{project_id}.{dataset_id}.child_present_score` 
ORDER BY score DESC
, child_id
, present_id
''')

In [ ]:
# Take a look at the data (head gives first 5 records)
childPresentScores.head()

In [ ]:
# Array of present ids -> count of presents allocated - used to keep count of how many times a present has been allocated.
# Every time a present is allocated, the stock_level is decremented down, until there are no more of a given present to allocate.
pa = b.q(f'''
SELECT id AS present_id
, stock_level
FROM `{project_id}.{dataset_id}.present` 
ORDER BY id
''')

pa.head()

In [ ]:
# Convert to dictionary (i.e. map)
presentAllocations = dict()
for index, row in pa.iterrows():
    presentId = row['present_id']
    stockLevel = row['stock_level']
    presentAllocations[presentId] = stockLevel

# Print out to demo how dictinaries work    
print(presentAllocations)  
print(presentAllocations[2])

In [ ]:
# Array of child ids -> allowed presents for the child - used to keep count of how many presents a child should be given for Xmas. 
# Every time a child is allocated a present, the value is decremented down, until there are no more presents to allocate to the child
ca = b.q(f'''
SELECT id AS child_id
, number_of_presents
FROM `{project_id}.{dataset_id}.child` 
ORDER BY id
''')

ca.head()

In [ ]:
# Convert to dictionary
childAllocations = dict()
for index, row in ca.iterrows():
    childAllocations[row["child_id"]] = row["number_of_presents"]

childAllocations 

In [ ]:
# Reset Dictionaries here, so you can repeatedly run this code! :-)
presentAllocations = dict()
for index, row in pa.iterrows():
    presentId = row['present_id']
    stockLevel = row['stock_level']
    presentAllocations[presentId] = stockLevel

childAllocations = dict()
for index, row in ca.iterrows():
    childAllocations[row["child_id"]] = row["number_of_presents"]

# Keep child/present allocations in array
allocations = [];

# # Loop through each of the child scores (highest scores processed first)
for index, childPresentScore in childPresentScores.iterrows():
    
    # Collect variables from dataframe
    childId = childPresentScore["child_id"]
    presentId = childPresentScore["present_id"]
    score = childPresentScore["score"]    

    # If there is a negative child/present score, then we won't allocate the present
    # TODO - Naughty/Nice!
    excludePresent = score < 0;

    # Check to see if we've already given away all of the presents
    presentAllocationExceeded = presentAllocations[presentId] == 0;
    
    # Check to see if the child has already been given all of their presents
    childAllocationExceeded = childAllocations[childId] == 0 
    
    # Work out if we should be including this present as an allocation
    include = not excludePresent and not presentAllocationExceeded and not childAllocationExceeded 
        
    # If we're including, update counts accordingly
    if include:
        presentAllocations[presentId] = presentAllocations[presentId] - 1
        childAllocations[childId] = childAllocations[childId] - 1

    # Add the allocation information to the allocations table (this will include 
    # products that weren't allocated to a member, as well as those that were.
    allocation = Allocation(
        childId,
        presentId,    
        include,
        excludePresent,
        presentAllocationExceeded,
        childAllocationExceeded
    )

    if index == 0:
        print("Example Output:")
    
    if index < 5:
        print(allocation, presentAllocations[childId])
        
    if index == 5:
        print("...")    

    allocations.append(allocation)
     

In [ ]:
# Create a table to insert the allocation results into
createTableSql = f'''
CREATE TABLE IF NOT EXISTS `{project_id}.{dataset_id}.allocations` 
(
  child_id INT64 NOT NULL
, present_id INT64 NOT NULL
, include BOOL NOT NULL
, exclude_present BOOL NOT NULL
, present_allocation_exceeded BOOL NOT NULL
, child_allocation_exceeded BOOL NOT NULL
)
'''

queryJob = b.db.query(createTableSql)
queryJob.result() # Wait until table is created

In [ ]:
# Clear out the table (to make sure it's empty)
clearTableSql = f'''
TRUNCATE TABLE `{project_id}.{dataset_id}.allocations`
'''

queryJob = b.db.query(clearTableSql)
queryJob.result() # Wait until table is created

In [ ]:
# Check there's nothing there
a = b.q(f'''
SELECT COUNT(*) AS allocation_count
FROM `{project_id}.{dataset_id}.allocations` 
''')

a

In [ ]:
# Add the allocations array into BQ
df = pd.DataFrame([vars(a) for a in allocations])
df.columns = [    
    "child_id",
    "present_id",
    "include",
    "exclude_present",
    "present_allocation_exceeded",
    "child_allocation_exceeded",
]

df.head()

In [ ]:
# Insert the dataframe into the BQ table
table = b.db.get_table(f'''{project_id}.{dataset_id}.allocations''')

errors = b.db.insert_rows_from_dataframe(table, df)
for chunk in errors:
    print(f"encountered {len(chunk)} errors: {chunk}")

In [ ]:
# Check we've correctly inserted the data into the table
a = b.q(f'''
SELECT * 
FROM `{project_id}.{dataset_id}.allocations` 
ORDER BY child_id
''')

a.head()

In [ ]:
# Take a look at present alloctions for different children
a = b.q(f'''
SELECT a.child_id
, c.name AS child_name
, cps.child_dislike_present_types
, cps.child_wish_present_types
, c.number_of_presents
, c.naughty
, a.present_id
, p.description AS present_desription
, cps.present_types
, p.value AS present_value
, cps.br_excl_present_dislike_score
, cps.br_weight_present_wish_score
, cps.br_weight_present_diversity_score
, cps.br_weight_product_value_score
, cps.score
, a.include
, a.exclude_present
, a.present_allocation_exceeded
, a.child_allocation_exceeded
FROM `{project_id}.{dataset_id}.allocations` a 
JOIN `{project_id}.{dataset_id}.child` c 
    ON a.child_id = c.id
JOIN `{project_id}.{dataset_id}.present` p 
    ON a.present_id = p.id
JOIN `{project_id}.{dataset_id}.child_present_score` cps 
    ON a.present_id = cps.present_id 
    AND a.child_id = cps.child_id
WHERE TRUE
    AND a.child_id = 1 -- Change to the child you'd like to see allocations for
    AND a.include IS TRUE -- Comment this line out to see why the other presents weren't allocated
ORDER BY a.child_id
, cps.score DESC
, a.present_id
''')

a